In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("0000000000.ipynb")

# 1 用户存款行为预测

### 数据集

银行营销数据集

#### 背景介绍

金融机构为了在下一次营销活动寻找最佳的改进策略，探寻如何在未来的营销活动中发挥更大的效力。为了解决这个问题，数据分析师必须分析该银行最近开展的营销活动，通过之前活动的情况预测用户是否会参加之后的营销活动。

#### 特征介绍

##### 个人信息相关
1 - age : 年龄 (数值列)

2 - job : 职业的种类(类别列: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : 婚姻状况 (类别列: 'divorced','married','single','unknown')

4 - education : 受教育情况(类别列: 
'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: 是否信用违约 (类别列: 'no','yes','unknown')

6 - balance: 余额 (数值列)

7 - housing: 是否有住房贷款(类别列: 'no','yes','unknown')

8 - loan: 是否有个人贷款? (类别列: 'no','yes','unknown')



##### 与当前营销活动的最后一次联系：
9 - contact: 联系人通信类型 (类别列: 'cellular','telephone')

10 - month: 最后一次联系的月份 (类别列: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

11 - day: 最后一次联系在星期几？ (类别列: 'mon','tue','wed','thu','fri')

12 - duration: 最后一次联系的时长, 单位秒 (数值列). 

##### 其他的一些特征：
13 - campaign: 此活动期间和此客户的联系人数 (数值列)

14 - pdays: 上次活动中联系客户后经过的天数 (数值列; 999 表示近期无联系)

15 - previous: 此活动之前和此客户端执行的联系人数 (数值列)

16 - poutcome: 上一次营销活动的结果 (类别列: 'failure','nonexistent','success')




### 目标

##### 预测定期存款额度

17 - deposit: 是否会存款 (类别列: 'yes','no')


- 数据来源：https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset

## 1.1 环境导入和数据准备

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
rng_seed = 44

In [3]:
df=pd.read_csv("bank.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [4]:
from sklearn.preprocessing import LabelEncoder
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

print(len(x))
x_simple_prepared = pd.get_dummies(x.copy())

lbe =  LabelEncoder()
lbe.fit(y)
y = lbe.transform(y)
x_simple_prepared

11162


,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,59,2343,5,1042,1,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
1,56,45,5,1467,1,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
2,41,1270,5,1389,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,55,2476,5,579,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,54,184,5,673,2,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,1,20,257,1,-1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
11158,39,733,16,83,4,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
11159,32,29,19,156,2,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
11160,43,0,8,9,2,172,5,0,0,0,...,0,0,1,0,0,0,1,0,0,0


## 1.2 模型的训练和评估(20分)

_Type your answer here, replacing this text._

In [5]:
np.random.seed(rng_seed)
# 利用get_dummies()方法对x进行编码
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
x_simple_prepared = pd.get_dummies(x.copy())
# 利用LabelEncoder()方法对y进行编码
lbe =  LabelEncoder()
lbe.fit(y)
y = lbe.transform(y)

np.random.seed(rng_seed)
# 划分train, test数据集
x_train, x_test, y_train, y_test = train_test_split(x_simple_prepared, y, train_size=0.8, test_size=1-0.8, random_state=0)
#训练SVC模型
svc_model = SVC(random_state=0, probability=True)
# 进行预测
svc_simple_score = accuracy_score(y_test, svc_model.fit(x_train, y_train).predict(x_test))
# 训练 LR 模型
lr_model = LogisticRegression(random_state=0, max_iter=10000)
# 进行预测
lr_simple_score = accuracy_score(y_test, lr_model.fit(x_train, y_train).predict(x_test))

print('svc accuracy:', svc_simple_score)
print('lr accuracy:', lr_simple_score)

svc accuracy: 0.7357814599193909
lr accuracy: 0.8248992386923422


In [6]:
grader.check("p1_1")

p1_1 results: All test cases passed!

## 1.3 数据预处理和再训练(20分)

**数据预处理**可以参考的数据准备操作：https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing
* 针对`x_train`, `y_train`, `x_test`, `y_test`使用数据准备操作，但要保证
  * 不能增加或删减`x_test`和`y_test`的行，但可以对它们进行变换
  * 可以对`x_train`和`y_train`做任何操作

In [15]:

# 数据预处理
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
# 处理异常值

threshold=3
q1 = np.percentile(x_train, 25, axis=0)
q3 = np.percentile(x_test, 75, axis=0)
print(x_train.head())
# 计算IQR（四分位数范围）
iqr = q3 - q1
# 计算上下界
lower_bound = q1 - threshold * iqr
upper_bound = q3 + threshold * iqr
# 将异常值替换为缺失值，并进行插补
is_outlier = (x_train < lower_bound) | (x_train > upper_bound)
x_train_no_outlier = x_train.copy()
x_train_no_outlier[is_outlier] = np.nan
x_train_no_outlier = SimpleImputer(strategy='mean').fit_transform(x_train_no_outlier)
x_test_no_outlier = x_test.copy()
x_test_no_outlier[is_outlier] = np.nan
x_test_no_outlier = SimpleImputer(strategy='mean').fit_transform(x_test_no_outlier)

# 分离特征和标签，对标签做编码
x_train_no_outlier = pd.DataFrame(x_train_no_outlier )
x_test_no_outlier = pd.DataFrame(x_test_no_outlier)

# 对特征做编码
x_train_no_outlier = pd.get_dummies(x_train_no_outlier)
x_test_no_outlier = pd.get_dummies(x_test_no_outlier)
# 对特征做降维
pca = PCA(n_components=20)
pca.fit(x_train_no_outlier)
x_train_no_outlier = pca.transform(x_train_no_outlier)
x_test_no_outlier = pca.transform(x_test_no_outlier)
# 对特征做归一化
scaler = StandardScaler()
scaler.fit(x_train_no_outlier)
x_train_no_outlier = scaler.transform(x_train_no_outlier)
x_test_no_outlier = scaler.transform(x_test_no_outlier)
#对特征做特征工程
# 特征工程
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(degree=2)
# x_train_no_outlier = poly.fit_transform(x_train_no_outlier)
# x_test_no_outlier = poly.transform(x_test_no_outlier)
np.random.seed(rng_seed)
# 训练SVC模型，处理完数据后，再次训练，你需要得到一个svc_score的指标
svc_new_model=SVC(random_state=0, probability=True)
svc_score=accuracy_score(y_test, svc_new_model.fit(x_train_no_outlier, y_train).predict(x_test_no_outlier))
#y_prob = svc_new_model.predict_proba(x_test)
#训练LR模型
lr_new_model=LogisticRegression(random_state=0, max_iter=1000)
lr_score=accuracy_score(y_test, lr_new_model.fit(x_train_no_outlier, y_train).predict(x_test_no_outlier))
#y_prob1=lr_new_model.predict_proba(x_test)
print('svc accuracy:', svc_score)
print('lr accuracy:', lr_score)

      age  balance  day  duration  campaign  pdays  previous  job_admin.  \
6111   49      104    8       285         1     -1         0           0   
1149   59      514    3       261         1     -1         0           0   
2954   33      195   16       984         5     -1         0           0   
2458   58      687    7       221         1     99         2           0   
7504   33     1077   18       432         3    374         3           1   

      job_blue-collar  job_entrepreneur  ...  month_jun  month_mar  month_may  \
6111                0                 0  ...          0          0          1   
1149                0                 0  ...          0          0          0   
2954                1                 0  ...          0          0          0   
2458                0                 0  ...          0          0          0   
7504                0                 0  ...          0          0          1   

      month_nov  month_oct  month_sep  poutcome_failure 

In [8]:
grader.check("p1_2")

p1_2 results: All test cases passed!

# 2 逻辑回归分类器的实现

## 2.1 确定优化目标(10分)

我们采用二元交叉熵作为损失函数，具体的推理过程可参考主课[Lec09](../../Lectures/Lec09/)

$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{y}^{(i)}) + (1 - y^{(i)}) \log(1 - \hat{y}^{(i)}) \right]$

* $J(\theta)$是损失函数
* $m$是样本数量
* $y^{(i)}$是第$i$个样本的真实标签
* $\hat{y}^{(i)}$是预测概率
* $\theta$是权重向量

In [9]:
np.random.seed(rng_seed)
# 实现损失函数
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_loss(y, y_hat):
    epsilon = 1e-15  # 一个很小的 epsilon 值，用于避免取零的对数
    #print(y_hat)
    logistic_loss = -np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    logistic_loss = logistic_loss / len(y)
    
    return logistic_loss

def loss_function(X, y, weights):
    

    z = np.dot(X, weights)
    y_hat = sigmoid(z)
    loss = logistic_loss(y, y_hat)
    return loss

In [10]:
grader.check("p2_1")

p2_1 results: All test cases passed!

## 2.2 计算优化目标的梯度(10分)
请根据上述目标计算梯度

$\nabla_{\theta} J(\theta) = \frac{1}{m} X^T (\hat{y} - y)$

请根据给出的公式，编写梯度函数。注意 $\hat{y}$ 要通过sigmoid函数后与 $y$ 相减

In [11]:
np.random.seed(rng_seed)
# 实现梯度函数
def gradient(X, y, weights):
    z = np.dot(X, weights)
    y_hat = sigmoid(z)
    gradient = np.dot(X.T, y_hat - y) / X.shape[0]
    return gradient

In [12]:
grader.check("p2_2")

p2_2 results: All test cases passed!

## 2.3 使用随机梯度下降（SGD）进行优化(10分)

权重更新的数学表示为：

$\theta = \theta - \alpha \nabla_{\theta} J(\theta)$

In [13]:
def stochastic_gradient_descent(X, y, weights, learning_rate=0.01, num_iterations=100):
    m = y.size
    loss_history = []
    
    for i in range(num_iterations):
        for j in range(m):
            # 随机选择一个数据点
            idx = np.random.randint(m)
            X_i = X[idx, :].reshape(1, -1)
            y_i = y[idx]
            # 计算梯度并更新权重
            grad =gradient(X_i, y_i, weights)
            weights -= learning_rate * grad
            
            # 计算并记录损失，用于监控
            loss = loss_function(X, y, weights)
            if(loss==np.nan):
                print(i,j)
            loss_history.append(loss)
        
        # 每10次迭代打印一次损失值
        if (i+1) % 1 == 0:
            print(f"Iteration {i+1}/{num_iterations}, Loss: {loss:.4f}")
    
    return weights, loss_history

np.random.seed(rng_seed)
num_features = 6
num_samples = 138
dummy_weights = np.random.rand(num_features)
dummy_X = np.random.rand(num_samples, num_features)
dummy_y = np.random.randint(0, 2, num_samples)

initial_weights = np.zeros(num_features)

final_weights, loss_history = stochastic_gradient_descent(dummy_X, dummy_y, initial_weights)

Iteration 1/100, Loss: 0.6920
Iteration 2/100, Loss: 0.6907
Iteration 3/100, Loss: 0.6898
Iteration 4/100, Loss: 0.6904
Iteration 5/100, Loss: 0.6894
Iteration 6/100, Loss: 0.6887
Iteration 7/100, Loss: 0.6879
Iteration 8/100, Loss: 0.6870
Iteration 9/100, Loss: 0.6860
Iteration 10/100, Loss: 0.6853
Iteration 11/100, Loss: 0.6849
Iteration 12/100, Loss: 0.6847
Iteration 13/100, Loss: 0.6847
Iteration 14/100, Loss: 0.6844
Iteration 15/100, Loss: 0.6845
Iteration 16/100, Loss: 0.6841
Iteration 17/100, Loss: 0.6839
Iteration 18/100, Loss: 0.6835
Iteration 19/100, Loss: 0.6832
Iteration 20/100, Loss: 0.6831
Iteration 21/100, Loss: 0.6836
Iteration 22/100, Loss: 0.6823
Iteration 23/100, Loss: 0.6846
Iteration 24/100, Loss: 0.6823
Iteration 25/100, Loss: 0.6828
Iteration 26/100, Loss: 0.6811
Iteration 27/100, Loss: 0.6814
Iteration 28/100, Loss: 0.6816


Iteration 29/100, Loss: 0.6813
Iteration 30/100, Loss: 0.6817
Iteration 31/100, Loss: 0.6817
Iteration 32/100, Loss: 0.6806
Iteration 33/100, Loss: 0.6802
Iteration 34/100, Loss: 0.6820
Iteration 35/100, Loss: 0.6813
Iteration 36/100, Loss: 0.6842
Iteration 37/100, Loss: 0.6796
Iteration 38/100, Loss: 0.6810
Iteration 39/100, Loss: 0.6793
Iteration 40/100, Loss: 0.6791
Iteration 41/100, Loss: 0.6792
Iteration 42/100, Loss: 0.6803
Iteration 43/100, Loss: 0.6791
Iteration 44/100, Loss: 0.6799
Iteration 45/100, Loss: 0.6798
Iteration 46/100, Loss: 0.6789
Iteration 47/100, Loss: 0.6791
Iteration 48/100, Loss: 0.6788
Iteration 49/100, Loss: 0.6790
Iteration 50/100, Loss: 0.6814
Iteration 51/100, Loss: 0.6792
Iteration 52/100, Loss: 0.6781
Iteration 53/100, Loss: 0.6780
Iteration 54/100, Loss: 0.6778
Iteration 55/100, Loss: 0.6778
Iteration 56/100, Loss: 0.6778
Iteration 57/100, Loss: 0.6792
Iteration 58/100, Loss: 0.6809
Iteration 59/100, Loss: 0.6851
Iteration 60/100, Loss: 0.6781
Iteratio

In [14]:
grader.check("p2_3")

p2_3 results: All test cases passed!

## 2.4 训练与评测(10分)

* 使用上面实现的函数训练一个LR分类器
* 为了保证结果方便测试...

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_simple_prepared, y, train_size=0.8, test_size=1-0.8, random_state=0)
x_train = x_train.values
x_test = x_test.values

In [16]:
from sklearn.metrics import classification_report
def predict(X, weights):
    # 根据给定的权重，预测数据集X的标签
    z = np.dot(X, weights)
    z=sigmoid(z)
    predictions = (z >= 0.5).astype(int)
    return predictions  # 预测概率大于等于0.5视为类别1，否则为类别0
    
num_features = x_train.shape[1]
np.random.seed(rng_seed)
# 标准化x_train和x_test，公式如下
# x = (x-mean(x))/std(x)
x_train=(x_train-np.mean(x_train,axis=0))/(np.std(x_train,axis=0))
x_test=(x_test-np.mean(x_test,axis=0))/np.std(x_test,axis=0)
# weight初始化为0
weights = np.zeros(num_features)
learning_rate = 0.01
num_iterations = 10
final_weights, loss_history = stochastic_gradient_descent(x_train, y_train, weights, learning_rate, num_iterations)

predictions = predict(x_test, final_weights)

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

Iteration 1/10, Loss: 0.4242
Iteration 2/10, Loss: 0.4219
Iteration 3/10, Loss: 0.4258


In [ ]:
predictions = predict(x_test, final_weights)
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)
loss_history
print(report)
print(accuracy)

              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1205
           1       0.80      0.78      0.79      1028

    accuracy                           0.81      2233
   macro avg       0.81      0.81      0.81      2233
weighted avg       0.81      0.81      0.81      2233

0.8101209135691895


In [ ]:
grader.check("p2_4")

0.691986260605109

# 3 基于实验结果的分析与研讨

请对本次作业做出总结：

- (1) 你采用的**数据预处理操作**是否提升了模型的accuracy(5分)?如果有提高，请给出提高的具体数值，并分析提升的原因(15分)。
- (2) 针对复现的逻辑回归模型，它和直接使用sklearn的精度差距为多大(10分)？请分析其背后的原因(10分)。

<font color='green'>**答:**</font>
是的，对于SVM模型，提高了0.08,对于LR模型，提高了0.01  
提升的可能原因：
* 标准化之后去除了异常值，降低了异常值的影响。
* 为了在

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

These are some submission instructions.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)

AttributeError: module 'nbconvert' has no attribute 'pdf'